In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# clone YOLOv5 repository
#!git clone https://github.com/ultralytics/yolov5  # clone repo
!git clone https://github.com/double-vin/yolov5_attention
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5_attention'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 164 (delta 18), reused 18 (delta 9), pack-reused 129
Receiving objects: 100% (164/164), 850.42 KiB | 7.27 MiB/s, done.
Resolving deltas: 100% (48/48), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# install dependencies as necessary
!pip install -qr /content/yolov5_attention/requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="fXNvqJubpdjp1mf8IS6s")
project = rf.workspace("detection-axsgy").project("uav-ce0zg")
dataset = project.version(11).download("yolov5")

In [ ]:
%cd /content/yolov5_attention
%cat {dataset.location}/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat //content/yolov5_attention/models/yolov5s_c3ca.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5_attention/models/MFNet_small2.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  #The focus layer was created to reduce layers, parameters, FLOPS, and CUDA memory and improve forward and backward speed while minimizing the impact of mAP
  ## ch_in, ch_out, kernel, stride, padding, groups
  [[-1, 1, Focus, [256, 3]],  # 0-P1/2
   # Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)
   [-1, 1, Conv, [256, 3, 2]],  # 1-P2/4
   #It employs a CSPNet strategy to partition the feature map of the base layer into two parts
   #and then merges them through a cross-stage hierarchy. The use of a split and merge strategy allows for more gradient flow through the network.
   # ch_in, ch_out, number, shortcut, groups, expansion
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [256, 3, 2]],  # 7-P5/32
   #Spatial Pyramid Pooling (SPP) is a pooling layer that removes the fixed-size constraint of the network, i.e. a CNN does not require a fixed-size input image
   [-1, 1, SPP, [256, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [128, False]],  # 13
 #nn.Upsample(scale_factor=2, mode='nearest')
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [128, False]],  # 17 (P3/medium)

   [-1, 1, Conv, [128, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [128, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [128, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [128, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train model for first time
#%%time
%cd /content/Uav-11/
!python /content/yolov5_attention/train.py --img 320 --batch-size -1 --epochs 600 --data {dataset.location}/data.yaml --cfg /content/yolov5_attention/models/MFNet_small2.yaml --hyp /content/yolov5_attention/data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name MFNet_small2_results  --cache

/content/Uav-11
2023-07-30 14:44:09.828912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 14:44:10.999433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/yolov5_attention/models/MFNet_small2.yaml, data=/content/Uav-11/data.yaml, hyp=/content/yolov5_attention/data/hyps/hyp.scratch-high.yaml, epochs=600, batch_size=-1, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=MFNet_small2_results, exist_ok=False, quad=False, cos_lr=False, label_smoothi

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
#!python train.py --img 320 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name MFNet-small2_results  --cache  --resume "/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/last.pt"

2023-07-30 14:04:46.191930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 14:04:47.044547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=320, rect=False, resume=/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=MFNet-small2_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], 

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-11/valid/images

/content/yolov5
detect: weights=['/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt'], source=/content/yolov5/Uav-11/valid/images, data=data/coco128.yaml, imgsz=[320, 320], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 232 layers, 1615007 parameters, 0 gradients, 17.3 GFLOPs
image 1/365 /content/yolov5/Uav-11/valid/images/20210924_2_50166786_69046593_jpg.rf.c64bd0ee61b459712e472983e0879dff.jpg: 320x320 1 Drone, Done. (0.009s)
image 2/365 /content/yolov5/Uav-11/valid/images/20210924_2_50166786_69046593_jpg.rf.e38b69902f356dc9b4ef771c2f3483ed.jpg: 320x320 1 Drone, D

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

val: data=/content/yolov5/Uav-11/data.yaml, weights=['/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=study, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False

Running study_data_best.txt --imgsz 256...
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 232 layers, 1615007 parameters, 0 gradients, 17.3 GFLOPs
val: Scanning '/content/yolov5/Uav-11/valid/labels.cache' images and labels... 365 found, 0 missing, 0 empty, 0 corrupt: 100% 365/365 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 12/12 [00:05<00:00,  2.33it/s]
                 all        365        365        172          6      0.

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

val: data=/content/yolov5/Uav-11/data.yaml, weights=['/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=speed, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 232 layers, 1615007 parameters, 0 gradients, 17.3 GFLOPs
val: Scanning '/content/yolov5/Uav-11/valid/labels.cache' images and labels... 365 found, 0 missing, 0 empty, 0 corrupt: 100% 365/365 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 12/12 [00:03<00:00,  3.13it/s]
                 all        365        365        149         16      0.906       0.82      0.861      0.904      0.

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

val: data=/content/yolov5/Uav-11/data.yaml, weights=['/content/gdrive/MyDrive/Runs/MFNet_small2_results/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 232 layers, 1615007 parameters, 0 gradients, 17.3 GFLOPs
val: Scanning '/content/yolov5/Uav-11/valid/labels.cache' images and labels... 365 found, 0 missing, 0 empty, 0 corrupt: 100% 365/365 [00:00<?, ?it/s]
               Class     Images     Labels         TP         FP          P          R         F1     mAP@.5 mAP@.5:.95: 100% 12/12 [00:04<00:00,  2.98it/s]
                 all        365        365        174          9      0.942      0.951      0.947      0.963       0.6

In [ ]:
#%cd /content/yolov5/
#!python detect.py --weights runs/train/MFNet_small2_results/weights/best.pt --img 320 --conf 0.3 --source /content/yolov5/Uav-7/valid/images

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize


In [ ]:
#!python val.py --task study --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --task speed --weights runs/train/MFNet_small2_results2/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5

In [ ]:
#!python val.py --img 320 --weights runs/train/MFNet_small2_results2/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5

In [ ]:
!zip -r /content/MFNet_small2.zip /content/gdrive/MyDrive/Runs/
!zip -r /content/MFNet_small2.zip /content/yolov5/runs/
from google.colab import files
files.download("/content/MFNet_small2.zip")

updating: content/gdrive/MyDrive/Runs/ (stored 0%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/ (stored 0%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/labels.jpg (deflated 22%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/labels_correlogram.jpg (deflated 23%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/train_batch0.jpg (deflated 3%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/train_batch1.jpg (deflated 4%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/train_batch2.jpg (deflated 4%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/events.out.tfevents.1672207470.17e7b326123a.799.0 (deflated 84%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/hyp.yaml (deflated 44%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/opt.yaml (deflated 47%)
updating: content/gdrive/MyDrive/Runs/MFNet_small2_results/val_batch0_pred.jpg (deflated 6%)
updating: content/gdrive/MyDrive/Runs/MFNe

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>